In [1]:
import tensorflow as tf
from keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
import numpy as np

def load_images_from_directory(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (150, 150))  # Resize images to a fixed size
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

# Define paths to your dataset directories
real_images_dir = '/content/drive/MyDrive/Test/Real'
fake_images_dir = '/content/drive/MyDrive/Test/Fake'

# Load and preprocess real images
real_images, real_labels = load_images_from_directory(real_images_dir, label=0)

# Load and preprocess fake images
fake_images, fake_labels = load_images_from_directory(fake_images_dir, label=1)

# Concatenate real and fake images and shuffle them
all_images = np.concatenate((real_images, fake_images), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)

# Shuffle data
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

# Split data into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_split = int(train_ratio * len(all_images))
val_split = int((train_ratio + val_ratio) * len(all_images))

X_train = all_images[:train_split]
y_train = all_labels[:train_split]

X_val = all_images[train_split:val_split]
y_val = all_labels[train_split:val_split]

X_test = all_images[val_split:]
y_test = all_labels[val_split:]

# Preprocess pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [ ]:
from tensorflow.keras import models, layers

# Define CNN model architecture #
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Save the trained model
model.save('fake_image_classifier_model.h5')

###############################################################################################
# Example usage of the trained model




In [27]:
import numpy as np
import cv2
def preprocess_image(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Resize the image to a fixed size (e.g., 150x150 pixels)
    image = cv2.resize(image, (150, 150))

    # Convert the image to RGB color space (if it's in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Normalize pixel values to range [0, 1]
    image = image.astype('float32') / 255.0

    # Expand dimensions to add batch dimension
    image = np.expand_dims(image, axis=0)

    return image

def predict_image(image_path):
    model = tf.keras.models.load_model('fake_image_classifier_model.h5')
    image = preprocess_image(image_path)
    prediction = model.predict(image)
    if prediction[0][0] > 0.5:
        return "Fake"
    else:
        return "Real"

In [44]:
new_image_path = 'C:\\Users\\jjazz\\OneDrive\\Desktop\\zfake\\testfake\\testfake\\real_00002.jpg'  # Replace with the path to your new image
prediction = predict_image(new_image_path)
print(f'Prediction for {new_image_path}: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Prediction for C:\Users\jjazz\OneDrive\Desktop\zfake\testfake\testfake\real_00002.jpg: Fake


In [ ]:
from google.colab import drive
drive.mount('/content/drive')